In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from matplotlib import cm
#import itertools
from numpy.linalg import inv
import random
import math

## WZ term to be calculated 

In [2]:
def eig_(M, sortby = 'real'):
    """ Returns the eigenvalues, eigenvectors (left and right if the matrix is
    non-symmetric) of a matrix M in an ascending order. sortby = 'real', 'imag' or 'abs'. """
    if np.allclose(M, M.T.conj()):
        evals, evecs = la.eig(M)
        sort_p = (evals.real).argsort()
        evals=evals[sort_p]
        evecs=evecs[:, sort_p]
        return evals, evecs
    else:
        evals, evecs_l, evecs_r = la.eig(M, left = True, right = True)
        if sortby == 'real':
           sort_p = (evals.real).argsort()
        elif sortby == 'imag':
           sort_p = (evals.imag).argsort()
        elif sortby == 'abs':
           sort_p = (np.abs(evals)).argsort()
        evals = evals[sort_p]
        evecs_r = evecs_r[:, sort_p]
        evecs_l = evecs_l[:, sort_p]
        return evals, evecs_l, evecs_r
    

In [3]:
# WZ integral for various bloch matrices depending on lambdas 
sigma_z = np.zeros((4,4), dtype=np.complex128)
sigma_z[0,0] += 1
sigma_z[1,1] += 1
sigma_z[2,2] += -1
sigma_z[3,3] += -1

# Bloch Hamiltonian
n_k=23 # number of steps 
k_x = np.linspace(0,2*np.pi,n_k, endpoint=False)
k_y = np.linspace(0,2*np.pi,n_k, endpoint=False)
k_z = np.linspace(0,2*np.pi,n_k, endpoint=False)
gamma=0.5
lamb=1
H = np.zeros((4,4), dtype=np.complex128)
n_t = n_k # number of steps 
t = np.linspace(0,1,n_t)

sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j],[1j,0]])
sigma_z = np.array([[1,0],[0,-1]])
I = np.identity(2)



def Bloch_pauli(kx, ky, kz, gamma, lamb):
    H_1 = 1j*lamb*np.sin(ky)*np.kron(sigma_x,I)+1j*(gamma+lamb*np.cos(ky))*np.kron(sigma_y,I)
    H_2 = 1j*lamb*np.sin(kx)*np.kron(sigma_z,I)+(gamma+lamb*np.cos(kx))*np.kron(I,sigma_z)
    H_3 = lamb*np.sin(kz)*np.kron(I,sigma_y)+(gamma+lamb*np.cos(kz))*np.kron(I,sigma_x)
    H_p = H_1+H_2+H_3
    #Hermitized-------------
    Hermi=(H_p+H_p.conj().T)/2
    #-----------------------
    return H_p


In [18]:
H = np.zeros((4,4), dtype=np.complex128)

def Bloch(kx, ky, kz, t, gamma, lamb):
    H_1 = 1j*lamb*np.sin(ky)*np.kron(sigma_x,I)+1j*(gamma+lamb*np.cos(ky))*np.kron(sigma_y,I)
    H_2 = 1j*lamb*np.sin(kx)*np.kron(sigma_z,I)+(gamma+lamb*np.cos(kx))*np.kron(I,sigma_z)
    H_3 = lamb*np.sin(kz)*np.kron(I,sigma_y)+(gamma+lamb*np.cos(kz))*np.kron(I,sigma_x)
    H = H_1+H_2+H_3
    H_WZ = H*(1-t)+H_const*t
    return H_WZ

def H1(kx, ky, kz, t, gamma, lamb):
    H_1 = 1j*lamb*np.cos(kx)*np.kron(sigma_z,I)+(-lamb*np.sin(kx))*np.kron(I,sigma_z)
    return H_1

def H2(kx, ky, kz, t, gamma, lamb):
    H_2 = 1j*lamb*np.cos(ky)*np.kron(sigma_x,I)+1j*(-lamb*np.sin(ky))*np.kron(sigma_y,I)
    return H_2

def H3(kx, ky, kz, t, gamma, lamb):
    H_1 = 1j*lamb*np.sin(ky)*np.kron(sigma_x,I)+1j*(gamma+lamb*np.cos(ky))*np.kron(sigma_y,I)
    H_2 = 1j*lamb*np.sin(kx)*np.kron(sigma_z,I)+(gamma+lamb*np.cos(kx))*np.kron(I,sigma_z)
    H_3 = lamb*np.sin(kz)*np.kron(I,sigma_y)+(gamma+lamb*np.cos(kz))*np.kron(I,sigma_x)
    H = H_1+H_2+H_3
    dt_H_WZ = - H + H_const
    return dt_H_WZ


In [19]:
integ_t = 0
integ = 0
for m in range(n_k):
    for j in range(n_k):
        for i in range(n_k):
            #--------------------------------------------matrix multiplcation for the matrices for trace----
            kx=k_x[i]
            ky=k_y[j]
            kz=np.pi #k_z[j]
            
            Hinv = inv(Bloch(kx, ky, kz,t[m],gamma,lamb))
            first0 = np.matmul(Hinv,H1(kx, ky, kz, t[m], gamma, lamb))
            second0 = np.matmul(Hinv,H2(kx, ky, kz,t[m], gamma, lamb))
            third0 = np.matmul(Hinv,H3(kx, ky, kz, t[m], gamma, lamb))
            first_two0 = np.matmul(first0,second0)
            last_multi0 = np.matmul(first_two0,third0) 
            
            first1 = np.matmul(Hinv,H1(kx, ky, kz, t[m], gamma, lamb))
            second1 = np.matmul(Hinv,H3(kx, ky, kz, t[m], gamma, lamb))
            third1 = np.matmul(Hinv,H2(kx, ky, kz, t[m], gamma, lamb))
            first_two1 = np.matmul(first1,second1)
            last_multi1 = np.matmul(first_two1,third1) 
            
            first2 = np.matmul(Hinv,H2(kx, ky, kz, t[m], gamma, lamb))
            second2 = np.matmul(Hinv,H1(kx, ky, kz, t[m], gamma, lamb))
            third2 = np.matmul(Hinv,H3(kx, ky, kz, t[m], gamma, lamb))
            first_two2 = np.matmul(first2,second2)
            last_multi2 = np.matmul(first_two2,third2) 
            
            first3 = np.matmul(Hinv,H2(kx, ky, kz, t[m], gamma, lamb))
            second3 = np.matmul(Hinv,H3(kx, ky, kz, t[m], gamma, lamb))
            third3 = np.matmul(Hinv,H1(kx, ky, kz, t[m], gamma, lamb))
            first_two3 = np.matmul(first3,second3)
            last_multi3 = np.matmul(first_two3,third3) 
                   
            first4 = np.matmul(Hinv,H3(kx, ky, kz, t[m], gamma, lamb))
            second4 = np.matmul(Hinv,H1(kx, ky, kz, t[m], gamma, lamb))
            third4 = np.matmul(Hinv,H2(kx, ky, kz, t[m], gamma, lamb))
            first_two4 = np.matmul(first4,second4)
            last_multi4 = np.matmul(first_two4,third4) 
            
            first5 = np.matmul(Hinv,H3(kx, ky, kz, t[m], gamma, lamb))
            second5 = np.matmul(Hinv,H2(kx, ky, kz, t[m], gamma, lamb))
            third5 = np.matmul(Hinv,H1(kx, ky, kz,t[m], gamma, lamb))
            first_two5 = np.matmul(first5,second5)
            last_multi5 = np.matmul(first_two5,third5) 
            #print('last matrix for trace:', last_multi5)
            #-------------------------------------# trace of the multiplied matrices ------------------------
            epsilon0 = 1
            epsilon1 = -1
            epsilon2 = -1
            epsilon3 = 1
            epsilon4 = 1
            epsilon5 = -1
            trace0 = epsilon0*np.trace(last_multi0)
            trace1 = epsilon1*np.trace(last_multi1)
            trace2 = epsilon2*np.trace(last_multi2)
            trace3 = epsilon3*np.trace(last_multi3)
            trace4 = epsilon4*np.trace(last_multi4)
            trace5 = epsilon5*np.trace(last_multi5)
            #print('each trace:', trace0,trace1,trace2)
            trace = trace0 + trace1 + trace2 + trace3 + trace4 + trace5
            #print(trace)
            #-----------------------------------------------------------------------------------------------
            integ_t += trace

integ = integ_t*(2*np.pi)**2/(24*np.pi**2*n_k**3) # (2pi)^2/n_k^3; (2*np.pi)**2

print('whole WZ integral:',integ) 

#kz=0 -- 2.6314e-17-0.1092j; kz=pi -- -1.0364e-15-0.0374j


whole WZ integral: (-1.0364108134986116e-15-0.03746269214443573j)
